In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [16]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [5]:
# Determine the number of unique values in each column.
unique_val = application_df.nunique()
unique_val

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
app_type_count = application_df.APPLICATION_TYPE.value_counts()
app_type_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Determine which values to replace if counts are less than ...?
replace_application = list(app_type_count[app_type_count < 612].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
class_count = application_df.CLASSIFICATION.value_counts()
class_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1728        1
C6100        1
C2561        1
C1570        1
C1900        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [9]:
# Determine which values to replace if counts are less than ..?
replace_class = list(class_count[class_count < 4000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [10]:
# Generate our categorical variable lists
cat_list = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
cat_list

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [11]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[cat_list]))


# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(cat_list)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True,right_index=True)
application_df = application_df.drop(cat_list,1)
application_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,1,5000,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,1,5000,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,1,5000,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [20]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

In [19]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  12
hidden_nodes_layer2 = 4

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 12)                492       
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 52        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 5         
Total params: 549
Trainable params: 549
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [32]:
# Create a callback
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

In [35]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=300,callbacks=[cp_callback])

Epoch 1/300
 97/804 [==>...........................] - ETA: 1s - loss: 0.5536 - accuracy: 0.7281
Epoch 00001: saving model to checkpoints\weights.01.hdf5
171/804 [=====>........................] - ETA: 1s - loss: 0.5428 - accuracy: 0.7378
Epoch 00001: saving model to checkpoints\weights.01.hdf5
299/804 [==========>...................] - ETA: 0s - loss: 0.5389 - accuracy: 0.7409
Epoch 00001: saving model to checkpoints\weights.01.hdf5
375/804 [============>.................] - ETA: 0s - loss: 0.5392 - accuracy: 0.7398
Epoch 00001: saving model to checkpoints\weights.01.hdf5
467/804 [================>.............] - ETA: 0s - loss: 0.5434 - accuracy: 0.7374
Epoch 00001: saving model to checkpoints\weights.01.hdf5
598/804 [=====================>........] - ETA: 0s - loss: 0.5436 - accuracy: 0.7368
Epoch 00001: saving model to checkpoints\weights.01.hdf5
698/804 [=========================>....] - ETA: 0s - loss: 0.5448 - accuracy: 0.7352
Epoch 00001: saving model to checkpoints\weights.01

140/804 [====>.........................] - ETA: 2s - loss: 0.5416 - accuracy: 0.7420
Epoch 00014: saving model to checkpoints\weights.14.hdf5
231/804 [=======>......................] - ETA: 1s - loss: 0.5436 - accuracy: 0.7405
Epoch 00014: saving model to checkpoints\weights.14.hdf5
329/804 [===========>..................] - ETA: 1s - loss: 0.5400 - accuracy: 0.7417
Epoch 00014: saving model to checkpoints\weights.14.hdf5
437/804 [===============>..............] - ETA: 1s - loss: 0.5421 - accuracy: 0.7406
Epoch 00014: saving model to checkpoints\weights.14.hdf5
546/804 [===================>..........] - ETA: 0s - loss: 0.5435 - accuracy: 0.7389
Epoch 00014: saving model to checkpoints\weights.14.hdf5
643/804 [======================>.......] - ETA: 0s - loss: 0.5440 - accuracy: 0.7379
Epoch 00014: saving model to checkpoints\weights.14.hdf5
725/804 [==========================>...] - ETA: 0s - loss: 0.5448 - accuracy: 0.7361
Epoch 00014: saving model to checkpoints\weights.14.hdf5
804/80

293/804 [=========>....................] - ETA: 1s - loss: 0.5434 - accuracy: 0.7358
Epoch 00027: saving model to checkpoints\weights.27.hdf5
392/804 [=============>................] - ETA: 1s - loss: 0.5447 - accuracy: 0.7346
Epoch 00027: saving model to checkpoints\weights.27.hdf5
483/804 [=================>............] - ETA: 0s - loss: 0.5471 - accuracy: 0.7323
Epoch 00027: saving model to checkpoints\weights.27.hdf5
579/804 [====================>.........] - ETA: 0s - loss: 0.5463 - accuracy: 0.7339
Epoch 00027: saving model to checkpoints\weights.27.hdf5
692/804 [========================>.....] - ETA: 0s - loss: 0.5466 - accuracy: 0.7338
Epoch 00027: saving model to checkpoints\weights.27.hdf5
794/804 [============================>.] - ETA: 0s - loss: 0.5451 - accuracy: 0.7352
Epoch 00027: saving model to checkpoints\weights.27.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5448 - accuracy: 0.7355
Epoch 28/300
 64/804 [=>............................] - ETA

412/804 [==============>...............] - ETA: 0s - loss: 0.5495 - accuracy: 0.7324
Epoch 00040: saving model to checkpoints\weights.40.hdf5
543/804 [===================>..........] - ETA: 0s - loss: 0.5463 - accuracy: 0.7346
Epoch 00040: saving model to checkpoints\weights.40.hdf5
625/804 [======================>.......] - ETA: 0s - loss: 0.5444 - accuracy: 0.7361
Epoch 00040: saving model to checkpoints\weights.40.hdf5
740/804 [==========================>...] - ETA: 0s - loss: 0.5462 - accuracy: 0.7351
Epoch 00040: saving model to checkpoints\weights.40.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5449 - accuracy: 0.7362
Epoch 41/300
 33/804 [>.............................] - ETA: 1s - loss: 0.5508 - accuracy: 0.7244
Epoch 00041: saving model to checkpoints\weights.41.hdf5
108/804 [===>..........................] - ETA: 1s - loss: 0.5436 - accuracy: 0.7396
Epoch 00041: saving model to checkpoints\weights.41.hdf5
208/804 [======>.......................] - ETA

471/804 [================>.............] - ETA: 0s - loss: 0.5413 - accuracy: 0.7409
Epoch 00053: saving model to checkpoints\weights.53.hdf5
564/804 [====================>.........] - ETA: 0s - loss: 0.5431 - accuracy: 0.7380
Epoch 00053: saving model to checkpoints\weights.53.hdf5
663/804 [=======================>......] - ETA: 0s - loss: 0.5455 - accuracy: 0.7351
Epoch 00053: saving model to checkpoints\weights.53.hdf5
761/804 [===========================>..] - ETA: 0s - loss: 0.5442 - accuracy: 0.7358
Epoch 00053: saving model to checkpoints\weights.53.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5450 - accuracy: 0.7352
Epoch 54/300
 68/804 [=>............................] - ETA: 1s - loss: 0.5593 - accuracy: 0.7169
Epoch 00054: saving model to checkpoints\weights.54.hdf5
155/804 [====>.........................] - ETA: 1s - loss: 0.5495 - accuracy: 0.7319
Epoch 00054: saving model to checkpoints\weights.54.hdf5
257/804 [========>.....................] - ETA

...

609/804 [=====================>........] - ETA: 0s - loss: 0.5432 - accuracy: 0.7359
Epoch 00242: saving model to checkpoints\weights.242.hdf5
728/804 [==========================>...] - ETA: 0s - loss: 0.5422 - accuracy: 0.7375
Epoch 00242: saving model to checkpoints\weights.242.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5441 - accuracy: 0.7360
Epoch 243/300
 25/804 [..............................] - ETA: 1s - loss: 0.5702 - accuracy: 0.7250
Epoch 00243: saving model to checkpoints\weights.243.hdf5
114/804 [===>..........................] - ETA: 1s - loss: 0.5465 - accuracy: 0.7322
Epoch 00243: saving model to checkpoints\weights.243.hdf5
216/804 [=======>......................] - ETA: 1s - loss: 0.5474 - accuracy: 0.7329
Epoch 00243: saving model to checkpoints\weights.243.hdf5
318/804 [==========>...................] - ETA: 1s - loss: 0.5477 - accuracy: 0.7320
Epoch 00243: saving model to checkpoints\weights.243.hdf5
428/804 [==============>...............

559/804 [===================>..........] - ETA: 0s - loss: 0.5449 - accuracy: 0.7386
Epoch 00255: saving model to checkpoints\weights.255.hdf5
683/804 [========================>.....] - ETA: 0s - loss: 0.5435 - accuracy: 0.7386
Epoch 00255: saving model to checkpoints\weights.255.hdf5
783/804 [============================>.] - ETA: 0s - loss: 0.5439 - accuracy: 0.7371
Epoch 00255: saving model to checkpoints\weights.255.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5437 - accuracy: 0.7371
Epoch 256/300
 69/804 [=>............................] - ETA: 1s - loss: 0.5467 - accuracy: 0.7301
Epoch 00256: saving model to checkpoints\weights.256.hdf5
168/804 [=====>........................] - ETA: 1s - loss: 0.5405 - accuracy: 0.7379
Epoch 00256: saving model to checkpoints\weights.256.hdf5
278/804 [=========>....................] - ETA: 0s - loss: 0.5381 - accuracy: 0.7413
Epoch 00256: saving model to checkpoints\weights.256.hdf5
354/804 [============>.................

525/804 [==================>...........] - ETA: 0s - loss: 0.5429 - accuracy: 0.7368
Epoch 00268: saving model to checkpoints\weights.268.hdf5
599/804 [=====================>........] - ETA: 0s - loss: 0.5418 - accuracy: 0.7380
Epoch 00268: saving model to checkpoints\weights.268.hdf5
729/804 [==========================>...] - ETA: 0s - loss: 0.5429 - accuracy: 0.7377
Epoch 00268: saving model to checkpoints\weights.268.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5440 - accuracy: 0.7367
Epoch 269/300
  1/804 [..............................] - ETA: 2s - loss: 0.5919 - accuracy: 0.6562
Epoch 00269: saving model to checkpoints\weights.269.hdf5
126/804 [===>..........................] - ETA: 1s - loss: 0.5369 - accuracy: 0.7408
Epoch 00269: saving model to checkpoints\weights.269.hdf5
198/804 [======>.......................] - ETA: 1s - loss: 0.5414 - accuracy: 0.7369
Epoch 00269: saving model to checkpoints\weights.269.hdf5
328/804 [===========>..................

473/804 [================>.............] - ETA: 0s - loss: 0.5478 - accuracy: 0.7339
Epoch 00281: saving model to checkpoints\weights.281.hdf5
579/804 [====================>.........] - ETA: 0s - loss: 0.5456 - accuracy: 0.7355
Epoch 00281: saving model to checkpoints\weights.281.hdf5
650/804 [=======================>......] - ETA: 0s - loss: 0.5442 - accuracy: 0.7362
Epoch 00281: saving model to checkpoints\weights.281.hdf5
779/804 [============================>.] - ETA: 0s - loss: 0.5435 - accuracy: 0.7365
Epoch 00281: saving model to checkpoints\weights.281.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5441 - accuracy: 0.7363
Epoch 282/300
 70/804 [=>............................] - ETA: 1s - loss: 0.5487 - accuracy: 0.7321
Epoch 00282: saving model to checkpoints\weights.282.hdf5
156/804 [====>.........................] - ETA: 1s - loss: 0.5437 - accuracy: 0.7400
Epoch 00282: saving model to checkpoints\weights.282.hdf5
268/804 [=========>....................

502/804 [=================>............] - ETA: 0s - loss: 0.5377 - accuracy: 0.7413
Epoch 00294: saving model to checkpoints\weights.294.hdf5
594/804 [=====================>........] - ETA: 0s - loss: 0.5385 - accuracy: 0.7420
Epoch 00294: saving model to checkpoints\weights.294.hdf5
723/804 [=========================>....] - ETA: 0s - loss: 0.5428 - accuracy: 0.7376
Epoch 00294: saving model to checkpoints\weights.294.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5439 - accuracy: 0.7372
Epoch 295/300
  1/804 [..............................] - ETA: 2s - loss: 0.4468 - accuracy: 0.7812
Epoch 00295: saving model to checkpoints\weights.295.hdf5
 94/804 [==>...........................] - ETA: 1s - loss: 0.5416 - accuracy: 0.7377
Epoch 00295: saving model to checkpoints\weights.295.hdf5
220/804 [=======>......................] - ETA: 1s - loss: 0.5448 - accuracy: 0.7388
Epoch 00295: saving model to checkpoints\weights.295.hdf5
294/804 [=========>....................

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Changed 2nd layer activation function to tanh

268/268 - 0s - loss: 0.5564 - accuracy: 0.7250
Loss: 0.5563626885414124, Accuracy: 0.7250145673751831


In [34]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Changed number of nodes from 7 to 12 in first layer

268/268 - 0s - loss: 0.5580 - accuracy: 0.7250
Loss: 0.5579537153244019, Accuracy: 0.7250145673751831


In [36]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Changed number of nodes from 150 to 300

268/268 - 0s - loss: 0.5577 - accuracy: 0.7256
Loss: 0.5577102303504944, Accuracy: 0.7255976796150208


In [40]:
# Export to HDF5 file
nn.save("AlphabetSoupCharityOptimization.h5")